In [1]:
#https://github.com/thomascmann/flatten-data
import copy
from collections.abc import MutableMapping, MutableSequence

def flatten_data(data, simple=1):
    out = []

    def flatten(x, name='',subgroup=None):
        if subgroup is None:
            subgroup = [{}]
        if isinstance(x, MutableMapping):
            for a in x:
                subgroup = flatten(x[a], ''.join([name, str(a), "_"]), subgroup)
        elif isinstance(x, MutableSequence):
            new_dicts = []
            for a in x:
                for dic in subgroup:
                    newdict = flatten(a, name, copy.deepcopy([dic]))
                    for i in newdict:
                        new_dicts.append(i)
            subgroup = new_dicts   
        else:
            for dic in subgroup:
                dic[name[:-1]] = x
        return subgroup

    if simple == 1:
        formatted_data = flatten(data)
        for i in formatted_data:
            out.append(i)
    else:
        for record in data:
            if isinstance(data, MutableSequence):
                formatted_data = flatten(record)
            elif isinstance(data, MutableMapping):
                formatted_data = flatten(data[record])
                for i in formatted_data:
                    i["_ID_VAR_"] = record

            for i in formatted_data:
                out.append(i)
    return out

In [2]:
!pip install flatten_json
!pip install facepy

In [3]:
from facepy import GraphAPI
import pandas as pd
from pandas.io.json import json_normalize
from flatten_json import flatten #https://medium.com/@amirziai/flatten-json-on-python-package-index-pypi-9e4951693a5a
# Initialize the Graph API with a valid access token (optional,
# but will allow you to do all sorts of fun stuff).
graph = GraphAPI('YOUR_TOKEN')



In [93]:
import time
#added limit at the end of post query. 100 seems to be max for each call
#&after= was added to iteration
query='germany'
places=graph.post('search?method=GET&type=place&q='+str(query)+'&fields=name,location,rating_count,category_list&center=52.5500361,13.3372237&limit=90')
pager=places['paging']['next'][places['paging']['next'].find('after=')+len('after='):]
#pager=places_tmp['paging']['cursors']['after']
if 'headers' in places:
    del places['headers']
if 'paging' in places:
    del places['paging']
flatdf=json_normalize(flatten_data(places))
#https://stackoverflow.com/questions/45918559/python-merging-rows-with-same-value-in-one-column
cat_df_id=flatdf.groupby('data_id').data_category_list_name.apply(lambda x : x.tolist()).to_frame()['data_category_list_name'].apply(pd.Series).add_prefix('Category_').fillna(' ')
#delete categories columns
flatdf = flatdf.drop(['data_category_list_id', 'data_category_list_name'], axis=1)
flatdf=flatdf.drop_duplicates()
#merge datasets
result=pd.merge(flatdf,cat_df_id, left_on='data_id', right_on='data_id')
print('Initial result shape',result.shape)
i=0
n=20
#for limits see this post. It's a little old but makes sense.
#https://stackoverflow.com/questions/14092989/facebook-api-4-application-request-limit-reached
for i in range(i,n):
    print(result['data_id'].unique().shape)
    if result.shape[0] <= 500:
        places_tmp=graph.post('search?method=GET&type=place&q='+str(query)+'&fields=name,location,rating_count,category_list&limit=90&after='+str(pager))
        #print(places_tmp['paging']['cursors']) #KeyError: 'after' means there are no more pages
        #print(pager)
        try:
            #print(places_tmp['paging']['cursors']['after'])
            #pager=places_tmp['paging']['next'][places_tmp['paging']['next'].find('after=')+len('after='):]
            pager=places_tmp['paging']['cursors']['after'] 
            del places_tmp['headers']
            del places_tmp['paging']
            flatdf_tmp=json_normalize(flatten_data(places_tmp))
            cat_tmp=flatdf_tmp.groupby('data_id').data_category_list_name.apply(lambda x : x.tolist()).to_frame()['data_category_list_name'].apply(pd.Series).add_prefix('Category_').fillna(' ')
            flatdf_tmp = flatdf_tmp.drop(['data_category_list_id', 'data_category_list_name'], axis=1)
            result_tmp=pd.merge(flatdf_tmp,cat_tmp, left_on='data_id', right_on='data_id')
            result=result.append(result_tmp)
            #print(result['data_id'].unique().shape)
        except: 
            print('end', i)
            break
    else:
        time.sleep(600)

    
    
    


Initial result shape (90, 13)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(174,)
(232,)
(297,)


KeyboardInterrupt: 

In [94]:
result['data_id'].unique().shape[0]

297

In [68]:
#tests
places=graph.get('search?method=GET&type=place&q=berlin&fields=name,location,rating_count,category_list&center=52.5500361,13.3372237&limit=100')
places['paging']['next'][places['paging']['next'].find('after=')+len('after='):]


'OTkZD'

In [95]:
result.shape

(552, 14)

In [96]:
result

,Category_0,Category_1,Category_2,Category_3,data_id,data_location_city,data_location_country,data_location_latitude,data_location_located_in,data_location_longitude,data_location_street,data_location_zip,data_name,data_rating_count
0,Country,,,,279516318736353,NaN,NaN,51,NaN,10,NaN,NaN,Alemania,NaN
1,Region,City,City,,213549102002249,Berlin,Germany,52.518391,NaN,13.401251,NaN,NaN,Berlín,NaN
2,City,National Park,,,187343588090963,NaN,NaN,52.516380914216,NaN,13.378899151666,NaN,NaN,"Brandenburg Gate, Berlin, Germany",8.0
3,Region,,,,327812700574720,NaN,NaN,52.395474014562,NaN,13.518712790547,NaN,NaN,"Brandenburg, Germany",NaN
4,City,,,,602289129838094,NaN,NaN,52.4985949,NaN,13.3847854,NaN,NaN,"Berlin, Germany - Europe",11.0
5,Region,,,,345774065433032,NaN,NaN,48.894107570617,NaN,11.583000803261,NaN,NaN,"Bayern, Germany",NaN
6,Airport Terminal,City,,,320746585017122,Berlin,Germany,52.552082365401,NaN,13.291771015536,andel's,NaN,"Tegel Airport, Berlin, Germany",26.0
7,Region,,,,278456442208672,NaN,NaN,48.430394114111,NaN,9.2463729858642,NaN,NaN,"Baden-Württemberg, Germany",NaN
8,City,City,,,220328594660274,Tiergarten,Germany,52.5167,NaN,13.3667,NaN,NaN,"Tiergarten, Berlin, Germany",NaN
9,Region,,,,252621764809436,NaN,NaN,51.41939,NaN,7.404526,NaN,NaN,"Nordrhein-Westfalen, Germany",NaN


In [99]:
r1= result.drop_duplicates()

In [100]:
r1.shape

(297, 14)

In [103]:
print('There are {} cities, {} countries' .format(r1['data_location_city'].unique().shape[0],r1['data_location_country'].unique().shape[0]))



There are 128 cities, 2 countries


In [104]:
r1['data_location_city'].unique()

array([nan, 'Berlin', 'Tiergarten', 'Prenzlauer Berg', 'Kreuzberg',
       'Hamburg', 'Munich', 'Frankfurt', 'Cologne', 'Bogenhausen',
       'Dresden', 'Stuttgart', 'Wilmersdorf', 'Hanover', 'Leipzig',
       'Potsdam', 'Steglitz', 'Bremen', 'Neukölln', 'Roßlau',
       'Schöneberg', 'Bamberg', 'Lichtenrade', 'Mannheim',
       'Märkisches Viertel', 'Oldenburg', 'Hagen', 'Bremerhaven', 'Belin',
       'Siegen', 'Heidelberg', 'Bonn', 'Bielefeld', 'Magdeburg', 'Trier',
       'Hildesheim', 'Cottbus-Schmellwitz', 'Kassel', 'Augsburg',
       'Erfurt', 'Aachen', 'Regensburg', 'Düsseldorf', 'Koblenz', 'Mainz',
       'Karlsruhe', 'Eisleben', 'Wuppertal', 'Ingolstadt', 'Ulm',
       'Pankow', 'Paderborn', 'Tessin', 'Plötzensee', 'Krefeld', 'Barlin',
       'Chemnitz', 'Bochum', 'Rostock', 'Wolfsburg', 'Karow', 'Strasburg',
       'Gera', 'Dessau', 'Fichtenwalde', 'Luckau', 'Bottrop', 'Wolfen',
       'Greifswald', 'Weißensee', 'Oberhausen', 'Gelsenkirchen',
       'Marzahn', 'Ahlen', 'Nurem

In [105]:
r1['data_location_country'].unique()

array([nan, 'Germany'], dtype=object)